In [25]:
# SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import os
from pyspark.sql.functions import *
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

spark = SparkSession.builder \
    .appName("lakehouse") \
    .config("spark.delta.columnMapping.mode", "name") \
    .config("spark.sql.catalogImplementation","hive") \
    .config("spark.sql.warehouse.dir","s3a://hive/") \
    .config("spark.sql.hive.metastore.version","3.1.3") \
    .config("spark.sql.hive.metastore.jars","path") \
    .config("spark.sql.hive.metastore.jars.path","file:///opt/spark/hive/jars/*") \
    .config("spark.sql.legacy.charVarcharAsString", True) \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic") \
    .config("hive.metastore.warehouse.dir","s3a://hive/") \
    .config("spark.hive.metastore.schema.verification","false") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .enableHiveSupport() \
    .getOrCreate()

# Minimierung des LOGS
spark.sparkContext.setLogLevel("ERROR")
log4jLogger = spark._jvm.org.apache.log4j
logger = log4jLogger.LogManager.getLogger("LOGGER")
logger.setLevel(log4jLogger.Level.INFO)
spark.sql("USE fidus_and")

DataFrame[]

In [26]:
# APK - kpi#10 Anzahl_Arzt-Patienten-Kontakte
df = spark.sql("""
WITH
  bh_data AS (
    SELECT
      KSPATNR AS patnr,
      KSBEHDAT AS datum,
      arzt AS schein_arzt,
      row_number() OVER (PARTITION BY KSPATNR, KSBEHDAT ORDER BY arzt) AS rn
    FROM behdatei
    WHERE arzt <> ''
  ),
  rp_data AS (
    SELECT
      rk.PATNR AS patnr,
      rp.LEISTDATUM AS datum,
      rp.arzt AS rechnung_arzt,
      row_number() OVER (PARTITION BY rk.PATNR, rp.LEISTDATUM ORDER BY rp.arzt) AS rn
    FROM rechkopf rk
    JOIN rechpos rp ON rk.RKOPFID = rp.RKOPFID
    WHERE rp.arzt <> ''
  ),
  ckey AS (
    -- Hier wird der erste gefundene Standort ermittelt
    SELECT first(SCHLUESSEL) AS Standort
    FROM ckeytabs
    WHERE OBJEKT = 'OSG_STANDORT'
  )

SELECT
  month(e.datum) AS Monat,
  weekofyear(e.datum) AS KW,
  year(e.datum) AS Jahr,
  e.datum AS Datum,
  CASE
    WHEN p.VERST = 8 THEN 'Privat'
    ELSE 'Kasse'
  END AS Versicherungsstatus,
  CASE
    WHEN (bh.schein_arzt IS NULL AND rp.rechnung_arzt IS NULL) THEN ckey.Standort
    WHEN bh.schein_arzt IS NULL THEN right(rp.rechnung_arzt, 3)
    ELSE right(bh.schein_arzt, 3)
  END AS BSNR,
  count(*) AS Anzahl,
  date_format(current_date(), 'yyyy-MM-dd') AS Lieferdatum,
  weekofyear(current_date()) AS LieferKW
FROM ekktexte e
LEFT JOIN patdat p ON p.PATNR = e.PATNR
LEFT JOIN (
  SELECT patnr, datum, schein_arzt
  FROM bh_data
  WHERE rn = 1
) bh ON e.patnr = bh.patnr AND e.datum = bh.datum
LEFT JOIN (
  SELECT patnr, datum, rechnung_arzt
  FROM rp_data
  WHERE rn = 1
) rp ON e.patnr = rp.patnr AND e.datum = rp.datum
CROSS JOIN ckey
WHERE
  (e.fduschl IN ('IVOMMDR:', 'IVOMMDL:', 'OPDKARR:', 'OPDKARL:', 'OPDAARR:', 'OPDAARL:')
    OR e.fduschl LIKE 'MBefA%')
  AND e.datum > '2021-04-01'
  AND dayofweek(e.datum) NOT IN (1,7)
  AND e.datum BETWEEN add_months(current_date(), -3) AND current_date()
GROUP BY
  e.datum,
  CASE
    WHEN p.VERST = 8 THEN 'Privat'
    ELSE 'Kasse'
  END,
  CASE
    WHEN (bh.schein_arzt IS NULL AND rp.rechnung_arzt IS NULL) THEN ckey.Standort
    WHEN bh.schein_arzt IS NULL THEN right(rp.rechnung_arzt, 3)
    ELSE right(bh.schein_arzt, 3)
  END
ORDER BY
  e.datum,
  CASE
    WHEN p.VERST = 8 THEN 'Privat'
    ELSE 'Kasse'
  END,
  CASE
    WHEN (bh.schein_arzt IS NULL AND rp.rechnung_arzt IS NULL) THEN ckey.Standort
    WHEN bh.schein_arzt IS NULL THEN right(rp.rechnung_arzt, 3)
    ELSE right(bh.schein_arzt, 3)
  END;
""")
spark.sql("CREATE DATABASE IF NOT EXISTS fidus_kpi")
spark.sql("USE fidus_kpi")
df.write.format("delta").mode("overwrite").option("path", 's3a://silver/fidus/kpi/kpi_10_apk').saveAsTable("kpi_10_apk")
spark.read.format("delta").load("s3a://silver/fidus/kpi/kpi_10_apk").show(5)

In [27]:
# ANP - kpi#20 Anzahl_neue Patienten
spark.sql("USE fidus_and")
df = spark.sql("""
WITH
  -- Ermittlung der Basisdaten aus den drei Quellen (arbehdat, behdatei, rechkopf/rechpos)
  pats AS (
    SELECT KSPATNR, KSBEHDAT, ARZT
    FROM arbehdat
    WHERE KSBEHDAT > '2021-04-01'
      AND KSBEHDAT BETWEEN add_months(current_date(), -6) AND current_date()
    UNION ALL
    SELECT KSPATNR, KSBEHDAT, ARZT
    FROM behdatei
    WHERE KSBEHDAT > '2021-04-01'
      AND KSBEHDAT BETWEEN add_months(current_date(), -6) AND current_date()
    UNION ALL
    SELECT rk.PATNR AS KSPATNR, rp.LEISTDATUM AS KSBEHDAT, rp.ARZT
    FROM rechkopf rk
    JOIN rechpos rp ON rk.RKOPFID = rp.RKOPFID
    WHERE rp.LEISTDATUM > '2021-04-01'
      AND rp.LEISTDATUM BETWEEN add_months(current_date(), -6) AND current_date()
  ),
  -- Für jeden Patienten werden alle (distinct) Datensätze ermittelt und mittels Fensterfunktion der unmittelbar vorhergehende Zeitpunkt bestimmt
  pats_with_prev AS (
    SELECT
      KSPATNR,
      KSBEHDAT,
      ARZT,
      lag(KSBEHDAT) OVER (PARTITION BY KSPATNR ORDER BY KSBEHDAT) AS prev_date
    FROM (
      SELECT DISTINCT KSPATNR, KSBEHDAT, ARZT
      FROM pats
    ) t
  ),
  -- Filter: Nur jene Datensätze, bei denen der Jahresunterschied zwischen dem aktuellen Datum und dem vorherigen (oder einem Standardwert) größer als 10 ist.
  pats_filtered AS (
    SELECT *
    FROM pats_with_prev
    -- Hier wird vereinfacht der Unterschied der Kalenderjahre herangezogen.
    WHERE (year(KSBEHDAT) - year(coalesce(prev_date, to_date('2000-01-01')))) > 10
  ),
  -- Ermittlung eines konstanten Standorts aus ckeytabs (analog zum OUTER APPLY)
  ckey AS (
    SELECT first(SCHLUESSEL) AS Standort
    FROM ckeytabs
    WHERE OBJEKT = 'OSG_STANDORT'
  )

SELECT
  month(p.KSBEHDAT) AS Monat,
  weekofyear(p.KSBEHDAT) AS KW,
  year(p.KSBEHDAT) AS Jahr,
  p.KSBEHDAT AS Datum,
  -- Wenn kein ARZT-Wert vorhanden ist, wird der ermittelte ckey-Standort genutzt, ansonsten werden die letzten 3 Zeichen von ARZT extrahiert
  CASE
    WHEN p.ARZT IS NULL OR p.ARZT = '' THEN ckey.Standort
    ELSE right(p.ARZT, 3)
  END AS Standort,
  count(*) AS Anzahl,
  date_format(current_date(), 'yyyy-MM-dd') AS Lieferdatum,
  weekofyear(current_date()) AS LieferKW
FROM pats_filtered p
CROSS JOIN ckey
GROUP BY
  p.KSBEHDAT,
  CASE
    WHEN p.ARZT IS NULL OR p.ARZT = '' THEN ckey.Standort
    ELSE right(p.ARZT, 3)
  END
ORDER BY p.KSBEHDAT;
""")
#df.show(5)
spark.sql("USE fidus_kpi")
df.write.format("delta").mode("overwrite").option("path", 's3a://silver/fidus/kpi/kpi_20_anp').saveAsTable("kpi_20_anp")
spark.read.format("delta").load("s3a://silver/fidus/kpi/kpi_20_anp").show(5)

In [28]:
# OZK - kpi#50 OP_Zahlen_Kat-OPs
spark.sql("USE fidus_and")
df = spark.sql("""
WITH
  -- Ermittlung des ersten (nicht-leeren) ARZT-Werts aus behdatei (Schein)
  schein_cte AS (
    SELECT
      KSPATNR AS patnr,
      KSBEHDAT AS datum,
      arzt AS schein_arzt,
      row_number() OVER (PARTITION BY KSPATNR, KSBEHDAT ORDER BY arzt) AS rn
    FROM behdatei
    WHERE arzt <> ''
  ),
  -- Ermittlung des ersten (nicht-leeren) ARZT-Werts aus rechkopf/rechpos (Rechnung)
  rechnung_cte AS (
    SELECT
      rk.PATNR AS patnr,
      rp.LEISTDATUM AS datum,
      rp.arzt AS rechnung_arzt,
      row_number() OVER (PARTITION BY rk.PATNR, rp.LEISTDATUM ORDER BY rp.arzt) AS rn
    FROM rechkopf rk
    JOIN rechpos rp ON rk.RKOPFID = rp.RKOPFID
    WHERE rp.arzt <> ''
  ),
  -- Ermittlung des Standorts aus ckeytabs (nicht korreliert)
  bsnr_cte AS (
    SELECT first(SCHLUESSEL) AS Standort
    FROM ckeytabs
    WHERE OBJEKT = 'OSG_STANDORT'
  )

SELECT
  ekk.datum,
  CASE
    WHEN p.VERST = 8 THEN 'Privat'
    ELSE 'Kasse'
  END AS Versicherungsstatus,
  CASE
    WHEN (schein.schein_arzt IS NULL AND rechnung.rechnung_arzt IS NULL) THEN bsnr.Standort
    WHEN (schein.schein_arzt IS NULL) THEN right(rechnung.rechnung_arzt, 3)
    ELSE right(schein.schein_arzt, 3)
  END AS Standort,
  count(*) AS Anzahl,
  date_format(current_date(), 'yyyy-MM-dd') AS Lieferdatum,
  weekofyear(current_date()) AS LieferKW
FROM ekktexte ekk
LEFT JOIN patdat p ON ekk.PATNR = p.PATNR
-- Einbindung des Scheinwertes (behdatei)
LEFT JOIN (
  SELECT patnr, datum, schein_arzt
  FROM schein_cte
  WHERE rn = 1
) schein ON ekk.patnr = schein.patnr AND ekk.datum = schein.datum
-- Einbindung des Rechnungswertes (rechkopf/rechpos)
LEFT JOIN (
  SELECT patnr, datum, rechnung_arzt
  FROM rechnung_cte
  WHERE rn = 1
) rechnung ON ekk.patnr = rechnung.patnr AND ekk.datum = rechnung.datum
-- CROSS JOIN, da der Standort aus ckeytabs konstant ist
CROSS JOIN bsnr_cte bsnr
WHERE ekk.fduschl IN ('OPDKARR:', 'OPDKARL:')
  AND ekk.datum > '2021-04-01'
  AND ekk.datum BETWEEN add_months(current_date(), -3) AND current_date()
GROUP BY
  ekk.datum,
  CASE
    WHEN p.VERST = 8 THEN 'Privat'
    ELSE 'Kasse'
  END,
  CASE
    WHEN (schein.schein_arzt IS NULL AND rechnung.rechnung_arzt IS NULL) THEN bsnr.Standort
    WHEN (schein.schein_arzt IS NULL) THEN right(rechnung.rechnung_arzt, 3)
    ELSE right(schein.schein_arzt, 3)
  END
ORDER BY ekk.datum;
""")
#df.show(5)
spark.sql("USE fidus_kpi")
df.write.format("delta").mode("overwrite").option("path", 's3a://silver/fidus/kpi/kpi_50_ozk').saveAsTable("kpi_50_ozk")
spark.read.format("delta").load("s3a://silver/fidus/kpi/kpi_50_ozk").show(5)

In [29]:
# OZI - kpi#50 OP_Zahlen_IVOMs
spark.sql("USE fidus_and")
df = spark.sql("""
WITH
  -- Ermittlung des ersten (nicht-leeren) ARZT-Werts aus behdatei (Schein)
  schein_cte AS (
    SELECT
      KSPATNR AS patnr,
      KSBEHDAT AS datum,
      arzt AS schein_arzt,
      row_number() OVER (PARTITION BY KSPATNR, KSBEHDAT ORDER BY arzt) AS rn
    FROM behdatei
    WHERE arzt <> ''
  ),
  -- Ermittlung des ersten (nicht-leeren) ARZT-Werts aus rechkopf/rechpos (Rechnung)
  rechnung_cte AS (
    SELECT
      rk.PATNR AS patnr,
      rp.LEISTDATUM AS datum,
      rp.arzt AS rechnung_arzt,
      row_number() OVER (PARTITION BY rk.PATNR, rp.LEISTDATUM ORDER BY rp.arzt) AS rn
    FROM rechkopf rk
    JOIN rechpos rp ON rk.RKOPFID = rp.RKOPFID
    WHERE rp.arzt <> ''
  ),
  -- Ermittlung des Standorts aus ckeytabs (als konstanter Wert)
  bsnr_cte AS (
    SELECT first(SCHLUESSEL) AS Standort
    FROM ckeytabs
    WHERE OBJEKT = 'OSG_STANDORT'
  )

SELECT
  ekk.datum,
  CASE
    WHEN p.VERST = 8 THEN 'Privat'
    ELSE 'Kasse'
  END AS Versicherungsstatus,
  CASE
    WHEN (schein.schein_arzt IS NULL AND rechnung.rechnung_arzt IS NULL) THEN bsnr.Standort
    WHEN (schein.schein_arzt IS NULL) THEN right(rechnung.rechnung_arzt, 3)
    ELSE right(schein.schein_arzt, 3)
  END AS Standort,
  count(*) AS Anzahl,
  date_format(current_date(), 'yyyy-MM-dd') AS Lieferdatum,
  weekofyear(current_date()) AS LieferKW
FROM ekktexte ekk
LEFT JOIN patdat p ON ekk.PATNR = p.PATNR
-- Einbindung des Scheinwertes (behdatei)
LEFT JOIN (
  SELECT patnr, datum, schein_arzt
  FROM schein_cte
  WHERE rn = 1
) schein ON ekk.patnr = schein.patnr AND ekk.datum = schein.datum
-- Einbindung des Rechnungswertes (rechkopf/rechpos)
LEFT JOIN (
  SELECT patnr, datum, rechnung_arzt
  FROM rechnung_cte
  WHERE rn = 1
) rechnung ON ekk.patnr = rechnung.patnr AND ekk.datum = rechnung.datum
-- CROSS JOIN liefert den konstanten Standort
CROSS JOIN bsnr_cte bsnr
WHERE ekk.fduschl IN ('IVOMMDL:', 'IVOMMDR:')
  AND ekk.datum > '2021-04-01'
  AND ekk.datum BETWEEN add_months(current_date(), -3) AND current_date()
GROUP BY
  ekk.datum,
  CASE
    WHEN p.VERST = 8 THEN 'Privat'
    ELSE 'Kasse'
  END,
  CASE
    WHEN (schein.schein_arzt IS NULL AND rechnung.rechnung_arzt IS NULL) THEN bsnr.Standort
    WHEN (schein.schein_arzt IS NULL) THEN right(rechnung.rechnung_arzt, 3)
    ELSE right(schein.schein_arzt, 3)
  END
ORDER BY ekk.datum;
""")
spark.sql("USE fidus_kpi")
df.write.format("delta").mode("overwrite").option("path", 's3a://silver/fidus/kpi/kpi_50_ozi').saveAsTable("kpi_50_ozi")
spark.read.format("delta").load('s3a://silver/fidus/kpi/kpi_50_ozi').show(5)

In [30]:
# OZP - kpi#50 OP_Zahlen_PPVs
spark.sql("USE fidus_and")
spark.sql("""
WITH
  -- Ermittlung des ersten (nicht-leeren) ARZT-Werts aus behdatei ("Schein")
  schein_cte AS (
    SELECT
      KSPATNR AS patnr,
      KSBEHDAT AS datum,
      arzt AS schein_arzt,
      row_number() OVER (PARTITION BY KSPATNR, KSBEHDAT ORDER BY arzt) AS rn
    FROM behdatei
    WHERE arzt <> ''
  ),
  -- Ermittlung des ersten (nicht-leeren) ARZT-Werts aus rechkopf/rechpos ("Rechnung")
  rechnung_cte AS (
    SELECT
      rk.PATNR AS patnr,
      rp.LEISTDATUM AS datum,
      rp.arzt AS rechnung_arzt,
      row_number() OVER (PARTITION BY rk.PATNR, rp.LEISTDATUM ORDER BY rp.arzt) AS rn
    FROM rechkopf rk
    JOIN rechpos rp ON rk.RKOPFID = rp.RKOPFID
    WHERE rp.arzt <> ''
  ),
  -- Ermittlung eines konstanten Standortwerts aus ckeytabs
  bsnr_cte AS (
    SELECT first(SCHLUESSEL) AS Standort
    FROM ckeytabs
    WHERE OBJEKT = 'OSG_STANDORT'
  )

SELECT
  ekk.datum,
  CASE
    WHEN p.VERST = 8 THEN 'Privat'
    ELSE 'Kasse'
  END AS Versicherungsstatus,
  CASE
    WHEN (schein.schein_arzt IS NULL AND rechnung.rechnung_arzt IS NULL) THEN bsnr.Standort
    WHEN (schein.schein_arzt IS NULL) THEN right(rechnung.rechnung_arzt, 3)
    ELSE right(schein.schein_arzt, 3)
  END AS Standort,
  count(*) AS Anzahl,
  date_format(current_date(), 'yyyy-MM-dd') AS Lieferdatum,
  weekofyear(current_date()) AS LieferKW
FROM ekktexte ekk
LEFT JOIN patdat p ON ekk.PATNR = p.PATNR
-- Einbindung des Scheinwerts
LEFT JOIN (
  SELECT patnr, datum, schein_arzt
  FROM schein_cte
  WHERE rn = 1
) schein ON ekk.patnr = schein.patnr AND ekk.datum = schein.datum
-- Einbindung des Rechnungswerts
LEFT JOIN (
  SELECT patnr, datum, rechnung_arzt
  FROM rechnung_cte
  WHERE rn = 1
) rechnung ON ekk.patnr = rechnung.patnr AND ekk.datum = rechnung.datum
-- CROSS JOIN liefert den konstanten Standort
CROSS JOIN bsnr_cte bsnr
WHERE ekk.fduschl IN ('OPDAARL:', 'OPDAARR:')
  AND ekk.inhalt LIKE '%PP-Vitrektomie%'
  AND ekk.datum > '2021-04-01'
  AND ekk.datum BETWEEN add_months(current_date(), -3) AND current_date()
GROUP BY
  ekk.datum,
  CASE
    WHEN p.VERST = 8 THEN 'Privat'
    ELSE 'Kasse'
  END,
  CASE
    WHEN (schein.schein_arzt IS NULL AND rechnung.rechnung_arzt IS NULL) THEN bsnr.Standort
    WHEN (schein.schein_arzt IS NULL) THEN right(rechnung.rechnung_arzt, 3)
    ELSE right(schein.schein_arzt, 3)
  END
ORDER BY ekk.datum;
""")
spark.sql("USE fidus_kpi")
df.write.format("delta").mode("overwrite").option("path", 's3a://silver/fidus/kpi/kpi_50_ozp').saveAsTable("kpi_50_ozp")
df.show(5)

+----------+-------------------+--------+------+-----------+--------+
|     datum|Versicherungsstatus|Standort|Anzahl|Lieferdatum|LieferKW|
+----------+-------------------+--------+------+-----------+--------+
|2024-11-21|              Kasse|     017|     6| 2025-02-20|       8|
|2024-11-25|             Privat|     017|     1| 2025-02-20|       8|
|2024-11-25|              Kasse|     017|    15| 2025-02-20|       8|
|2024-11-28|              Kasse|     017|     4| 2025-02-20|       8|
|2024-12-02|              Kasse|     017|    15| 2025-02-20|       8|
+----------+-------------------+--------+------+-----------+--------+
only showing top 5 rows



In [31]:
# OZAO - kpi#50 OP_Zahlen_Andere OPs
spark.sql("USE fidus_and")
spark.sql("""
WITH
  -- Ermittlung des ersten (nicht-leeren) ARZT-Werts aus behdatei (Schein)
  schein_cte AS (
    SELECT
      KSPATNR AS patnr,
      KSBEHDAT AS datum,
      arzt AS schein_arzt,
      row_number() OVER (PARTITION BY KSPATNR, KSBEHDAT ORDER BY arzt) AS rn
    FROM behdatei
    WHERE arzt <> ''
  ),
  -- Ermittlung des ersten (nicht-leeren) ARZT-Werts aus rechkopf/rechpos (Rechnung)
  rechnung_cte AS (
    SELECT
      rk.PATNR AS patnr,
      rp.LEISTDATUM AS datum,
      rp.arzt AS rechnung_arzt,
      row_number() OVER (PARTITION BY rk.PATNR, rp.LEISTDATUM ORDER BY rp.arzt) AS rn
    FROM rechkopf rk
    JOIN rechpos rp ON rk.RKOPFID = rp.RKOPFID
    WHERE rp.arzt <> ''
  ),
  -- Ermittlung des Standorts aus ckeytabs (als konstanter Wert)
  bsnr_cte AS (
    SELECT first(SCHLUESSEL) AS Standort
    FROM ckeytabs
    WHERE OBJEKT = 'OSG_STANDORT'
  )

SELECT
  ekk.datum,
  CASE
    WHEN p.VERST = 8 THEN 'Privat'
    ELSE 'Kasse'
  END AS Versicherungsstatus,
  CASE
    WHEN (schein.schein_arzt IS NULL AND rechnung.rechnung_arzt IS NULL) THEN bsnr.Standort
    WHEN (schein.schein_arzt IS NULL) THEN right(rechnung.rechnung_arzt, 3)
    ELSE right(schein.schein_arzt, 3)
  END AS Standort,
  count(*) AS Anzahl,
  date_format(current_date(), 'yyyy-MM-dd') AS Lieferdatum,
  weekofyear(current_date()) AS LieferKW
FROM ekktexte ekk
LEFT JOIN patdat p ON ekk.PATNR = p.PATNR
-- Einbindung des Scheinwerts aus behdatei
LEFT JOIN (
  SELECT patnr, datum, schein_arzt
  FROM schein_cte
  WHERE rn = 1
) schein ON ekk.patnr = schein.patnr AND ekk.datum = schein.datum
-- Einbindung des Rechnungswerts aus rechkopf/rechpos
LEFT JOIN (
  SELECT patnr, datum, rechnung_arzt
  FROM rechnung_cte
  WHERE rn = 1
) rechnung ON ekk.patnr = rechnung.patnr AND ekk.datum = rechnung.datum
-- CROSS JOIN, da der Standort aus ckeytabs konstant ist
CROSS JOIN bsnr_cte bsnr
WHERE ekk.fduschl IN ('OPDAARL:', 'OPDAARR:')
  AND ekk.inhalt NOT LIKE '%PP-Vitrektomie%'
  AND ekk.datum > '2021-04-01'
  AND ekk.datum BETWEEN add_months(current_date(), -3) AND current_date()
GROUP BY
  ekk.datum,
  CASE
    WHEN p.VERST = 8 THEN 'Privat'
    ELSE 'Kasse'
  END,
  CASE
    WHEN (schein.schein_arzt IS NULL AND rechnung.rechnung_arzt IS NULL) THEN bsnr.Standort
    WHEN (schein.schein_arzt IS NULL) THEN right(rechnung.rechnung_arzt, 3)
    ELSE right(schein.schein_arzt, 3)
  END
ORDER BY ekk.datum;
""")
spark.sql("USE fidus_kpi")
df.write.format("delta").mode("overwrite").option("path", 's3a://silver/fidus/kpi/kpi_50_ozao').saveAsTable("kpi_50_ozao")
spark.read.format("delta").load("s3a://silver/fidus/kpi/kpi_50_ozao").show(5)

+----------+-------------------+--------+------+-----------+--------+
|     datum|Versicherungsstatus|Standort|Anzahl|Lieferdatum|LieferKW|
+----------+-------------------+--------+------+-----------+--------+
|2024-11-21|              Kasse|     017|     6| 2025-02-20|       8|
|2024-11-25|             Privat|     017|     1| 2025-02-20|       8|
|2024-11-25|              Kasse|     017|    15| 2025-02-20|       8|
|2024-11-28|              Kasse|     017|     4| 2025-02-20|       8|
|2024-12-02|              Kasse|     017|    15| 2025-02-20|       8|
+----------+-------------------+--------+------+-----------+--------+
only showing top 5 rows



In [32]:
# AZ --kpi#51 Anzahl_Zuweiser
spark.sql("USE fidus_and")
spark.sql("""
WITH
  -- Für die erste Arzt-Ermittlung (Schein)
  schein_cte AS (
    SELECT
      KSPATNR AS patnr,
      KSBEHDAT AS datum,
      arzt AS schein_arzt,
      row_number() OVER (PARTITION BY KSPATNR, KSBEHDAT ORDER BY arzt) AS rn
    FROM behdatei
    WHERE arzt <> ''
  ),
  -- Für die erste Arzt-Ermittlung aus rechkopf/rechpos (Rechnung)
  rechnung_cte AS (
    SELECT
      rk.PATNR AS patnr,
      rp.LEISTDATUM AS datum,
      rp.arzt AS rechnung_arzt,
      row_number() OVER (PARTITION BY rk.PATNR, rp.LEISTDATUM ORDER BY rp.arzt) AS rn
    FROM rechkopf rk
    JOIN rechpos rp ON rk.RKOPFID = rp.RKOPFID
    WHERE rp.arzt <> ''
  ),
  -- Konstanter Standort aus ckeytabs
  bsnr_cte AS (
    SELECT first(SCHLUESSEL) AS Standort
    FROM ckeytabs
    WHERE OBJEKT = 'OSG_STANDORT'
  ),
  -- Simulation des OUTER APPLY für den Arzt-Kontakt:
  -- Für jeden Eintrag in ekktexte (als Haupttabelle) wird der zuletzt erfasste Termin aus ekktexte ermittelt,
  -- bei dem fduschl wie 'MBefA%' gilt und der innerhalb des Zeitfensters liegt (zwischen 1 Monat vor und 1 Tag vor dem Hauptdatum).
  arzt_kontakt_cte AS (
    SELECT
      main.PATNR AS main_patnr,
      main.datum AS main_datum,
      ak.datum AS kontakt_datum,
      row_number() OVER (PARTITION BY main.PATNR, main.datum ORDER BY ak.datum DESC) AS rn
    FROM ekktexte main
    JOIN ekktexte ak
      ON main.PATNR = ak.PATNR
    WHERE ak.fduschl LIKE 'MBefA%'
      AND ak.datum BETWEEN add_months(main.datum, -1) AND date_sub(main.datum, 1)
  ),
  -- Hauptabfrage: Zusammenführung aller Informationen
  main_query AS (
    SELECT DISTINCT
      ekk.datum AS OP_Datum,
      ekk.patnr AS PatientID,
      COALESCE(a.ADRBSNR, '') AS Zuweiser_BSNR,
      CASE
         WHEN a.ADRNA2 IS NULL OR a.ADRNA2 = ''
         THEN trim(concat_ws(' ', COALESCE(a.adrtit, ''), COALESCE(a.ADRNAM, ''), COALESCE(a.ADRVOR, '')))
         ELSE a.adrna2
      END AS Zuweiser_Name,
      COALESCE(a.adrplz, '') AS Zuweiser_PLZ,
      -- Erste Arzt-Ermittlung: Wenn weder Schein noch Rechnung vorhanden sind, wird der konstante Standort verwendet.
      CASE
         WHEN (schein.schein_arzt IS NULL AND rechnung.rechnung_arzt IS NULL) THEN bsnr.Standort
         WHEN (schein.schein_arzt IS NULL) THEN right(rechnung.rechnung_arzt, 3)
         ELSE right(schein.schein_arzt, 3)
      END AS Standort,
      akontakt.kontakt_datum AS VorherigeArztKontaktDatum,
      -- Zweite Arzt-Ermittlung (bezogen auf den Kontakt-Termin)
      CASE
         WHEN (schein2.schein_arzt IS NULL AND rechnung2.rechnung_arzt IS NULL) THEN 'kein Arzt'
         WHEN (schein2.schein_arzt IS NULL) THEN right(rechnung2.rechnung_arzt, 3)
         ELSE right(schein2.schein_arzt, 3)
      END AS VorherigeArztKontaktStandort,
      date_format(current_date(), 'yyyy-MM-dd') AS Lieferdatum,
      weekofyear(current_date()) AS LieferKW
    FROM ekktexte ekk
    LEFT JOIN patinfo p2
      ON p2.PATNR = ekk.PATNR AND p2.schl = 'P_ZUW:'
    LEFT JOIN adrver a
      ON a.ADRNR = p2.INHALT
    -- Erste Arzt-Ermittlung (Schein)
    LEFT JOIN (
      SELECT patnr, datum, schein_arzt
      FROM schein_cte
      WHERE rn = 1
    ) schein
      ON ekk.patnr = schein.patnr AND ekk.datum = schein.datum
    -- Erste Arzt-Ermittlung (Rechnung)
    LEFT JOIN (
      SELECT patnr, datum, rechnung_arzt
      FROM rechnung_cte
      WHERE rn = 1
    ) rechnung
      ON ekk.patnr = rechnung.patnr AND ekk.datum = rechnung.datum
    -- Join für den Arzt-Kontakt (lateral simuliert)
    LEFT JOIN (
      SELECT main_patnr, main_datum, kontakt_datum
      FROM arzt_kontakt_cte
      WHERE rn = 1
    ) akontakt
      ON ekk.patnr = akontakt.main_patnr AND ekk.datum = akontakt.main_datum
    -- Zweite Arzt-Ermittlung, basierend auf dem Kontakt-Termin:
    LEFT JOIN (
      SELECT patnr, datum, schein_arzt
      FROM schein_cte
      WHERE rn = 1
    ) schein2
      ON ekk.patnr = schein2.patnr AND akontakt.kontakt_datum = schein2.datum
    LEFT JOIN (
      SELECT patnr, datum, rechnung_arzt
      FROM rechnung_cte
      WHERE rn = 1
    ) rechnung2
      ON ekk.patnr = rechnung2.patnr AND akontakt.kontakt_datum = rechnung2.datum
    -- Konstanter Standort
    CROSS JOIN bsnr_cte bsnr
    WHERE ekk.datum > '2021-04-01'
      AND ekk.fduschl IN ('OPDKARR:', 'OPDKARL:', 'IVOMMDL:', 'IVOMMDR:', 'OPDAARL:', 'OPDAARR:')
  )

SELECT *
FROM main_query
ORDER BY OP_Datum;
""")
spark.sql("USE fidus_kpi")
df.write.format("delta").mode("overwrite").option("path", 's3a://silver/fidus/kpi/kpi_51_az').saveAsTable("kpi_51_az")
spark.read.format("delta").load("s3a://silver/fidus/kpi/kpi_51_az").show(5)

+----------+-------------------+--------+------+-----------+--------+
|     datum|Versicherungsstatus|Standort|Anzahl|Lieferdatum|LieferKW|
+----------+-------------------+--------+------+-----------+--------+
|2024-11-21|              Kasse|     017|     6| 2025-02-20|       8|
|2024-11-25|             Privat|     017|     1| 2025-02-20|       8|
|2024-11-25|              Kasse|     017|    15| 2025-02-20|       8|
|2024-11-28|              Kasse|     017|     4| 2025-02-20|       8|
|2024-12-02|              Kasse|     017|    15| 2025-02-20|       8|
+----------+-------------------+--------+------+-----------+--------+
only showing top 5 rows

